In [2]:
# TODO
'case_id' #OK
'collision_date': str, #datetime.date,
'collision_severity': #OK
'collision_time': str, #time, 
'county_city_location': str, 
'hit_and_run': #OK
'jurisdiction': #OK
'lighting': #OK
'location_type': #OK
'officer_id': #OK
'pcf_violation': #OK
'pcf_violation_category': #OK,
'pcf_violation_subsection': #OK,
'population': #OK
'primary_collision_factor': #OK,
'process_date': str, #datetime.date,
'ramp_intersection': #OK
'road_condition_1': #OK,
'road_condition_2': #OK,
'road_surface': #OK,
'tow_away': #OK,
'type_of_collision': #OK,
'weather_1': #OK,
'weather_2': #OK,

SyntaxError: illegal target for annotation (<ipython-input-2-7ecb849568a0>, line 3)

In [4]:
import pandas as pd
import numpy as np
import datetime
import time

In [5]:
dtype_collisions={
    'case_id': np.uint64, # cannot use in tsince biggest case id is 9.870011e+18, np.uint64: 64-bit unsigned integer (from 0 to 2**64-1)
    'collision_date': str, #datetime.date,
    'collision_severity': str,# => convert to ID
    'collision_time': str, #time, 
    'county_city_location': str,
    'hit_and_run': str,# => convert to ID
    'jurisdiction': pd.Int16Dtype(),
    'lighting': str,
    'location_type': str,
    'officer_id': str,
    'pcf_violation': pd.Int64Dtype(),
    'pcf_violation_category': str,
    'pcf_violation_subsection': str,
    'population': pd.Int64Dtype(),
    'primary_collision_factor': str,
    'process_date': str, #datetime.date,
    'ramp_intersection': pd.Int64Dtype(),
    'road_condition_1': str,
    'road_condition_2': str,
    'road_surface': str,
    'tow_away': 'boolean',
    'type_of_collision': str,
    'weather_1': str,
    'weather_2': str, 
}#check what type to give for integers
#int does not allow nulls
#np.uint64 allows up to 64 bytes (necessary for case id)
#pd.Int16/32/64Dtype() allows nulls

In [6]:
collisions_data = pd.read_csv("CSV-2018/collisions2018.csv", dtype=dtype_collisions ,usecols=dtype_collisions.keys()) #

In [7]:
collisions_data

,case_id,collision_date,collision_severity,collision_time,county_city_location,hit_and_run,jurisdiction,lighting,location_type,officer_id,...,primary_collision_factor,process_date,ramp_intersection,road_condition_1,road_condition_2,road_surface,tow_away,type_of_collision,weather_1,weather_2
0,1,2002-01-18,property damage only,15:30:00,1900,not hit and run,9575,daylight,NaN,16418,...,vehicle code violation,2002-06-14,<NA>,normal,NaN,dry,True,rear end,clear,NaN
1,2,2002-02-13,property damage only,19:30:00,1500,misdemeanor,9420,NaN,NaN,14360,...,vehicle code violation,2002-03-29,<NA>,normal,NaN,dry,False,hit object,clear,NaN
2,3,2002-02-11,property damage only,15:30:00,1502,misdemeanor,9420,daylight,highway,16117,...,vehicle code violation,2003-07-15,<NA>,normal,NaN,dry,False,hit object,clear,NaN
3,4,2002-02-12,property damage only,07:45:00,1500,misdemeanor,9420,daylight,NaN,6078,...,vehicle code violation,2002-05-22,<NA>,normal,NaN,dry,False,sideswipe,clear,NaN
4,5,2002-02-14,severe injury,11:35:00,1500,not hit and run,9420,daylight,NaN,13851,...,vehicle code violation,2003-07-28,<NA>,normal,NaN,dry,True,overturned,clear,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3678058,9870011224092016011,2001-12-24,pain,09:20:00,3600,not hit and run,9870,daylight,highway,16011,...,vehicle code violation,2002-04-26,7,normal,NaN,dry,True,hit object,clear,NaN
3678059,9870011224123011850,2001-12-24,property damage only,12:30:00,3600,not hit and run,9870,daylight,intersection,11850,...,vehicle code violation,2002-04-26,5,normal,NaN,dry,False,sideswipe,clear,NaN
3678060,9870011226102009803,2001-12-26,other injury,10:20:00,3600,not hit and run,9870,daylight,NaN,9803,...,vehicle code violation,2002-03-29,8,normal,NaN,dry,True,head-on,clear,NaN
3678061,9870011228210011458,2001-12-28,property damage only,21:00:00,3600,misdemeanor,9870,dark with no street lights,NaN,11458,...,vehicle code violation,2002-03-08,8,normal,NaN,dry,False,hit object,cloudy,NaN


In [8]:
clean_data = collisions_data.copy()

In [9]:
translation_collision_severity ={
    "fatal": 1,
    "severe injury": 2,
    "other injury": 3,
    "pain": 4,
    "property damage only": 0,
}

translation_hit_and_run ={
    "felony": "F",
    "misdemeanor": "M",
    "not hit and run": "N",
}

translation_lighting ={
    "daylight": "A",
    "dusk or dawn": "B",
    "dark with street lights": "C",
    "dark with no street lights": "D",
    "dark with street lights not functioning": "E",
}

translation_location_type ={
    "highway": "H",
    "intersection": "I",
    "ramp": "R",
}

translation_pcf_violation_category ={
    'unknown': 0,
    'dui': 1,
    'impeding traffic': 2,
    'speeding': 3,
    'following too closely': 4,
    'wrong side of road': 5,
    'improper passing': 6,
    'unsafe lane change': 7,
    'improper turning': 8,
    'automobile right of way':9,
    'pedestrian right of way': 10,
    'pedestrian violation': 11,
    'traffic signals and signs': 12,
    'hazardous parking': 13,
    'lights': 14,
    'brakes': 15,
    'other equipment': 16,
    'other hazardous violation': 17,
    'other than driver (or pedestrian)': 18,
    'unsafe starting or backing': 19,
    'other improper driving': 22,
    'pedestrian dui': 23,
    'fell asleep': 24,    
}

translation_primary_collision_factor ={
    "vehicle code violation": "A",
    "other improper driving": "B",
    "other than driver": "C",
    "unknown": "D",
    "fell asleep": "E",
}

translation_road_condition ={
    'holes': "A",
    'loose material': "B",
    'obstruction': "C",
    'construction': "D",
    'reduced width':"E",
    'flooded': "F",
    'other':"G",
    'normal': "H",
}

translation_road_surface ={
    'dry': "A",
    'wet': "B",
    'snowy': "C", 
    'slippery': "D", 
}

translation_type_of_collision ={
    'head-on': "A",
    'sideswipe':"B",
    'rear end':"C",
    'broadside':"D",
    'hit object':"E",
    'overturned':"F",
    'pedestrian':"G",
    'other':"H",
}

translation_weather ={
    'clear': "A", 
    'cloudy': "B",
    'raining': "C",
    'snowing':"D",
    'fog': "E",
    'other': "F", 
    'wind':"G",
}

In [10]:
clean_data["collision_severity"] = collisions_data["collision_severity"].map(lambda s: translation_collision_severity[s]).astype(pd.Int16Dtype())
clean_data["hit_and_run"] = collisions_data["hit_and_run"].map(lambda s: translation_hit_and_run.get(s, None))
clean_data["lighting"] = collisions_data["lighting"].map(lambda s: translation_lighting.get(s, None))
clean_data["location_type"] = collisions_data["location_type"].map(lambda s: translation_location_type.get(s, None))
clean_data["primary_collision_factor"] = collisions_data["primary_collision_factor"].map(lambda s: translation_primary_collision_factor.get(s, None))
clean_data["pcf_violation_category"] = collisions_data["pcf_violation_category"].map(lambda s: translation_pcf_violation_category.get(s, None)).astype(pd.Int16Dtype())
clean_data["road_condition_1"] = collisions_data["road_condition_1"].map(lambda s: translation_road_condition.get(s, None))
clean_data["road_condition_2"] = collisions_data["road_condition_2"].map(lambda s: translation_road_condition.get(s, None))
clean_data["road_surface"] = collisions_data["road_surface"].map(lambda s: translation_road_surface.get(s, None))
clean_data["type_of_collision"] = collisions_data["type_of_collision"].map(lambda s: translation_type_of_collision.get(s, None))
clean_data["weather_1"] = collisions_data["weather_1"].map(lambda s: translation_weather.get(s, None))
clean_data["weather_2"] = collisions_data["weather_2"].map(lambda s: translation_weather.get(s, None))

In [11]:
clean_data

,case_id,collision_date,collision_severity,collision_time,county_city_location,hit_and_run,jurisdiction,lighting,location_type,officer_id,...,primary_collision_factor,process_date,ramp_intersection,road_condition_1,road_condition_2,road_surface,tow_away,type_of_collision,weather_1,weather_2
0,1,2002-01-18,0,15:30:00,1900,N,9575,A,None,16418,...,A,2002-06-14,<NA>,H,None,A,True,C,A,None
1,2,2002-02-13,0,19:30:00,1500,M,9420,None,None,14360,...,A,2002-03-29,<NA>,H,None,A,False,E,A,None
2,3,2002-02-11,0,15:30:00,1502,M,9420,A,H,16117,...,A,2003-07-15,<NA>,H,None,A,False,E,A,None
3,4,2002-02-12,0,07:45:00,1500,M,9420,A,None,6078,...,A,2002-05-22,<NA>,H,None,A,False,B,A,None
4,5,2002-02-14,2,11:35:00,1500,N,9420,A,None,13851,...,A,2003-07-28,<NA>,H,None,A,True,F,A,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3678058,9870011224092016011,2001-12-24,4,09:20:00,3600,N,9870,A,H,16011,...,A,2002-04-26,7,H,None,A,True,E,A,None
3678059,9870011224123011850,2001-12-24,0,12:30:00,3600,N,9870,A,I,11850,...,A,2002-04-26,5,H,None,A,False,B,A,None
3678060,9870011226102009803,2001-12-26,3,10:20:00,3600,N,9870,A,None,9803,...,A,2002-03-29,8,H,None,A,True,A,A,None
3678061,9870011228210011458,2001-12-28,0,21:00:00,3600,M,9870,D,None,11458,...,A,2002-03-08,8,H,None,A,False,E,B,None


In [12]:
conditions = clean_data["road_surface"]

In [13]:
clean_data["ramp_intersection"].to_csv("test.csv",  index=False)

In [14]:
set(clean_data["pcf_violation_category"])

{0.0,
 1.0,
 nan,
 3.0,
 4.0,
 5.0,
 6.0,
 7.0,
 8.0,
 9.0,
 10.0,
 nan,
 12.0,
 13.0,
 nan,
 nan,
 11.0,
 17.0,
 18.0,
 19.0,
 16.0,
 15.0,
 22.0,
 14.0,
 24.0,
 23.0,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 2.0,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 na

###### Delete nrows ######

sep=',' by default 

header='0' by default (first line is column names)

should we use index_col=0 for collisions data

usecols to make sure we're reading correct columns

add dtype

use converters : Dict of functions for converting values in certain columns. Keys can either be integers or column labels.

??na_values / keep_default_na / na_filter

check with sql if need to infer_datetime_format/keep_date_col/date_parser/dayfirst/cache_dates

comment is it useful?

error_bad_lines/warn_bad_lines

In [ ]:
#read last line
# f1 = open("CSV-2018/parties2018.csv", "r")
# last_line = f1.readlines()[-1]
# print(last_line)
# f1.close()

7286606

In [ ]:
print(parties_data[7286605:]['case_id'])
print(parties_data[7286605:]['id'])
print(parties_data[7286605:]['party_age'])
print(parties_data[7286605:]['party_number'])

                                ------------------------------------------

In [ ]:
victims_data = pd.read_csv("CSV-2018/victims2018.csv", nrows=nrows) 

In [ ]:
f1 = open("CSV-2018/parties2018.csv", "r")
last_line = f1.readlines()[-1]
print(last_line)
f1.close()

In [ ]:
collisions_data = pd.read_csv("CSV-2018/collisions2018.csv", nrows=nrows) 

In [ ]:
# print(victims_data)

In [ ]:
# print(collisions_data)

In [ ]:
case_id = 5
parties_data_case = parties_data[parties_data["case_id"] == case_id]
victims_data_case = victims_data[victims_data["case_id"] == case_id]
collisions_data_case = collisions_data[collisions_data["case_id"] == case_id]

In [ ]:
parties_data_case

In [ ]:
victims_data_case

In [ ]:
collisions_data_case

In [ ]:
options = ["B", "C", "D", None]
parties_data_test = parties_data[~(parties_data["cellphone_use"].isin(options) | parties_data["cellphone_use"].isnull().values.any())]



In [ ]:
parties_data_test

In [ ]:
len(parties_data)